**Version Info**

In [ ]:
USE GolfNowOne
GO
;

SELECT
*
FROM [dbo].[dbverinfo]
ORDER BY EventDate DESC

**Orders/Min by type**

In [ ]:
USE GolfNowOne
GO

--Testing Locations
--fl_standard18(35650), fl_cody(35608), fl_standard9(35651)
--ld_loadtest1(38119), ld_loadtest2(38120), ld_loadtest3(38121)
--ld_loadtest4(38122), ld_loadtest5(38123), ld_loadtest6(38124), ld_loadtest7(38125)
--ld_loadtest8(38126), ld_loadtest9(38127), ld_loadtest10(38128)


-- Limit to automation 18 Holes location 
DECLARE @locationid AS INT = 38119;
DECLARE @fromdate AS SMALLDATETIME = '03-22-2023 23:00:00';
DECLARE @todate AS SMALLDATETIME = '03-23-2023 01:00:00';

/*
Count orders by minute created

Not converted to est yet - using local course time so need to update that at some point
*/
SELECT
    CONVERT(DATE, o.DateCreated) AS 'day'
    ,datepart(hh, o.datecreated) AS 'hour'
    ,datepart(mi, o.datecreated) AS 'minute'
    ,count(CASE WHEN o.statusid = 1 THEN 1 ELSE NULL END) AS 'open'
    ,count(CASE WHEN o.statusid = 2 THEN 1 ELSE NULL END) AS 'closed'
    ,count(CASE WHEN o.statusid = 3 THEN 1 ELSE NULL END) AS 'cancelled'
    ,count(CASE WHEN o.statusid = 4 THEN 1 ELSE NULL END) AS 'shipped'
    ,count(CASE WHEN o.statusid = 5 THEN 1 ELSE NULL END) AS 'partially paid'
    ,count(CASE WHEN o.statusid = 6 THEN 1 ELSE NULL END) AS 'voided'
    ,count(CASE WHEN o.statusid = 7 THEN 1 ELSE NULL END) AS 'paid'
    ,count(CASE WHEN o.statusid = 99 THEN 1 ELSE NULL END) AS 'abandoned'
FROM
    spa s WITH (nolock)
    LEFT JOIN [order] o WITH (nolock)
    ON o.SpaID = s.id
    LEFT JOIN OrderStatusLU oslu WITH (nolock)
    ON oslu.id = o.StatusID
WHERE
	o.datecreated > @fromdate
    AND o.datecreated < @todate
	AND s.id IN (38119,38120,38121,38122,38123,38124,38125,38126,38127,38128)
    --AND s.id = @locationid
GROUP BY
	CONVERT(DATE, o.DateCreated),
	datepart(hh, o.datecreated),
	datepart(mi, o.datecreated)
ORDER BY
	datepart(hh, o.datecreated) asc,
	datepart(mi, o.datecreated) asc

**Top 10 Orders/Min within timeframe**

In [ ]:
USE GolfNowOne
GO

DECLARE @fromdate AS SMALLDATETIME = '07-22-2023 00:00:00';
DECLARE @todate AS SMALLDATETIME = '07-22-2023 23:59:00';

/*
Count orders by minute created

Not converted to est yet - using local course time so need to update that at some point
*/
SELECT TOP 10
    CONVERT(DATE, o.DateCreated) AS 'day'
    ,datepart(hh, o.datecreated) AS 'hour'
    ,datepart(mi, o.datecreated) AS 'minute'
    ,count(CASE WHEN o.statusid = 1 THEN 1 ELSE NULL END) AS 'open'
    ,count(CASE WHEN o.statusid = 2 THEN 1 ELSE NULL END) AS 'closed'
    ,count(CASE WHEN o.statusid = 3 THEN 1 ELSE NULL END) AS 'cancelled'
    ,count(CASE WHEN o.statusid = 4 THEN 1 ELSE NULL END) AS 'shipped'
    ,count(CASE WHEN o.statusid = 5 THEN 1 ELSE NULL END) AS 'partially paid'
    ,count(CASE WHEN o.statusid = 6 THEN 1 ELSE NULL END) AS 'voided'
    ,count(CASE WHEN o.statusid = 7 THEN 1 ELSE NULL END) AS 'paid'
    ,count(CASE WHEN o.statusid = 99 THEN 1 ELSE NULL END) AS 'abandoned'
FROM
    spa s WITH (nolock)
    LEFT JOIN [order] o WITH (nolock)
    ON o.SpaID = s.id
    LEFT JOIN OrderStatusLU oslu WITH (nolock)
    ON oslu.id = o.StatusID
WHERE
	o.datecreated > @fromdate
    AND o.datecreated < @todate
GROUP BY
	CONVERT(DATE, o.DateCreated),
	datepart(hh, o.datecreated),
	datepart(mi, o.datecreated)
ORDER BY
	count(CASE WHEN o.statusid = 2 THEN 1 ELSE NULL END) desc

**Orders/Day**

In [ ]:
USE GolfNowOne
GO

/* Total Sales and closed orders by 2 hour timeslot per day

CONVERTED TO EASTERN - Note that you'll have some orders that occurred late on west coast that roll into the next morning EST

Note that spas that are NOW terminated (or other status) but used to be live are not included. Need to figure out including legit ones but there is a lot of junk in there (in the terminated list, that is) and needs a cleanup before being included
*/
SELECT
    TOP 100
    DATEPART(yyyy, outc.datecompletedest) 'year'
    ,DATEPART(mm, outc.datecompletedest) 'month'
    ,datepart(dd, outc.datecompletedest) 'day'
    ,DATENAME(dw, outc.datecompletedest)'dayOfWeek'
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 0 AND outc.SourceID = 4 THEN 1 ELSE NULL END) AS '0-2am EST Desktop'
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 0 AND outc.SourceID = 5 THEN 1 ELSE NULL END) AS '0-2am EST Tablet'
    --,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 0 AND outc.SourceID = 6 THEN 1 ELSE NULL END) AS '0-2am EST SmartPlay'
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 0 AND outc.SourceID = 8 THEN 1 ELSE NULL END) AS '0-2am EST GN Prepaid'
    --,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 0 AND outc.SourceID = 9 THEN 1 ELSE NULL END) AS '0-2am EST GN Featured'
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 1 AND outc.SourceID = 4  THEN 1 ELSE NULL END) AS '2-4 EST Desktop'
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 1 AND outc.SourceID = 5  THEN 1 ELSE NULL END) AS '2-4 EST Tablet'
    --,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 1 AND outc.SourceID = 6 THEN 1 ELSE NULL END) AS '2-4 EST SmartPlay'
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 1 AND outc.SourceID = 8 THEN 1 ELSE NULL END) AS '2-4 EST GN Prepaid'
    --,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 1 AND outc.SourceID = 9 THEN 1 ELSE NULL END) AS '2-4 EST GN Featured'     
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 2 AND outc.SourceID = 4  THEN 1 ELSE NULL END) AS '4-6 EST Desktop'
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 2 AND outc.SourceID = 5  THEN 1 ELSE NULL END) AS '4-6 EST Tablet'
    --,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 2 AND outc.SourceID = 6 THEN 1 ELSE NULL END) AS '4-6 EST SmartPlay'
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 2 AND outc.SourceID = 8 THEN 1 ELSE NULL END) AS '4-6 EST GN Prepaid'
    --,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 2 AND outc.SourceID = 9 THEN 1 ELSE NULL END) AS '4-6 EST GN Featured' 
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 3 AND outc.SourceID = 4  THEN 1 ELSE NULL END) AS '6-8 EST Desktop'
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 3 AND outc.SourceID = 5  THEN 1 ELSE NULL END) AS '6-8 EST Tablet'
    --,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 3 AND outc.SourceID = 6 THEN 1 ELSE NULL END) AS '6-8 EST SmartPlay'
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 3 AND outc.SourceID = 8 THEN 1 ELSE NULL END) AS '6-8 EST GN Prepaid'
    --,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 3 AND outc.SourceID = 9 THEN 1 ELSE NULL END) AS '6-8 EST GN Featured' 
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 4 AND outc.SourceID = 4  THEN 1 ELSE NULL END) AS '8-10 EST Desktop'
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 4 AND outc.SourceID = 5  THEN 1 ELSE NULL END) AS '8-10 EST Tablet'
    --,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 4 AND outc.SourceID = 6 THEN 1 ELSE NULL END) AS '8-10 EST SmartPlay'
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 4 AND outc.SourceID = 8 THEN 1 ELSE NULL END) AS '8-10 EST GN Prepaid'
    --,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 4 AND outc.SourceID = 9 THEN 1 ELSE NULL END) AS '8-10 EST GN Featured' 
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 5 AND outc.SourceID = 4  THEN 1 ELSE NULL END) AS '10-noon EST Desktop'
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 5 AND outc.SourceID = 5  THEN 1 ELSE NULL END) AS '10-noon EST Tablet'
    --,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 5 AND outc.SourceID = 6 THEN 1 ELSE NULL END) AS '10-noon EST SmartPlay'
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 5 AND outc.SourceID = 8 THEN 1 ELSE NULL END) AS '10-noon EST GN Prepaid'
    --,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 5 AND outc.SourceID = 9 THEN 1 ELSE NULL END) AS '10-noon EST GN Featured' 
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 6 AND outc.SourceID = 4  THEN 1 ELSE NULL END) AS 'noon-2 EST Desktop'
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 6 AND outc.SourceID = 5  THEN 1 ELSE NULL END) AS 'noon-2 EST Tablet'
    --,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 6 AND outc.SourceID = 6 THEN 1 ELSE NULL END) AS 'noon-2 EST SmartPlay'
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 6 AND outc.SourceID = 8 THEN 1 ELSE NULL END) AS 'noon-2 EST GN Prepaid'
    --,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 6 AND outc.SourceID = 9 THEN 1 ELSE NULL END) AS 'noon-2 EST GN Featured' 
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 7 AND outc.SourceID = 4  THEN 1 ELSE NULL END) AS '2-4 EST Desktop'
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 7 AND outc.SourceID = 5  THEN 1 ELSE NULL END) AS '2-4 EST Tablet'
    --,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 7 AND outc.SourceID = 6 THEN 1 ELSE NULL END) AS '2-4 EST SmartPlay'
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 7 AND outc.SourceID = 8 THEN 1 ELSE NULL END) AS '2-4 EST GN Prepaid'
    --,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 7 AND outc.SourceID = 9 THEN 1 ELSE NULL END) AS '2-4 EST GN Featured' 
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 8 AND outc.SourceID = 4  THEN 1 ELSE NULL END) AS '4-6 EST Desktop'
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 8 AND outc.SourceID = 5  THEN 1 ELSE NULL END) AS '4-6 EST Tablet'
    --,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 8 AND outc.SourceID = 6 THEN 1 ELSE NULL END) AS '4-6 EST SmartPlay'
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 8 AND outc.SourceID = 8 THEN 1 ELSE NULL END) AS '4-6 EST GN Prepaid'
    --,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 8 AND outc.SourceID = 9 THEN 1 ELSE NULL END) AS '4-6 EST GN Featured' 
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 9 AND outc.SourceID = 4  THEN 1 ELSE NULL END) AS '6-8 EST Desktop'
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 9 AND outc.SourceID = 5  THEN 1 ELSE NULL END) AS '6-8 EST Tablet'
    --,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 9 AND outc.SourceID = 6 THEN 1 ELSE NULL END) AS '6-8 EST SmartPlay'
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 9 AND outc.SourceID = 8 THEN 1 ELSE NULL END) AS '6-8 EST GN Prepaid'
    --,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 9 AND outc.SourceID = 9 THEN 1 ELSE NULL END) AS '6-8 EST GN Featured' 
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 10 AND outc.SourceID = 4  THEN 1 ELSE NULL END) AS '8-10 EST Desktop'
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 10 AND outc.SourceID = 5  THEN 1 ELSE NULL END) AS '8-10 EST Tablet'
    --,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 10 AND outc.SourceID = 6 THEN 1 ELSE NULL END) AS '8-10 EST SmartPlay'
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 10 AND outc.SourceID = 8 THEN 1 ELSE NULL END) AS '8-10 EST GN Prepaid'
    --,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 10 AND outc.SourceID = 9 THEN 1 ELSE NULL END) AS '8-10 EST GN Featured' 
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 11 AND outc.SourceID = 4  THEN 1 ELSE NULL END) AS '10-midnight EST Desktop'
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 11 AND outc.SourceID = 5  THEN 1 ELSE NULL END) AS '10-midnight EST Tablet'
    --,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 11 AND outc.SourceID = 6 THEN 1 ELSE NULL END) AS '10-midnight EST SmartPlay'
    ,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 11 AND outc.SourceID = 8 THEN 1 ELSE NULL END) AS '10-midnight EST GN Prepaid'
    --,count(CASE WHEN (datepart(hour, outc.datecompletedest) / 2) = 11 AND outc.SourceID = 9 THEN 1 ELSE NULL END) AS '10-midnight EST GN Featured'     
    ,count(outc.datecompletedest) 'totalOrderCount EST'
----sum(o.finaltotal) Amount /*Order $$$ Amount*/

--(count(case when (datepart(hour, outc.datecompletedest) / 2) = 0 then 1 else null end))/24 '0-2am EST per 5 min',
--(count(case when (datepart(hour, outc.datecompletedest) / 2) = 1 then 1 else null end))/24 '2-4 EST per 5 min',
--(count(case when (datepart(hour, outc.datecompletedest) / 2) = 2 then 1 else null end))/24 '4-6 EST per 5 min',
--(count(case when (datepart(hour, outc.datecompletedest) / 2) = 3 then 1 else null end))/24 '6-8 EST per 5 min',
--(count(case when (datepart(hour, outc.datecompletedest) / 2) = 4 then 1 else null end))/24 '8-10 EST per 5 min',
--(count(case when (datepart(hour, outc.datecompletedest) / 2) = 5 then 1 else null end))/24 '10-noon EST per 5 min',
--(count(case when (datepart(hour, outc.datecompletedest) / 2) = 6 then 1 else null end))/24 'noon-2 EST per 5 min',
--(count(case when (datepart(hour, outc.datecompletedest) / 2) = 7 then 1 else null end))/24 '2-4 EST per 5 min',
--(count(case when (datepart(hour, outc.datecompletedest) / 2) = 8 then 1 else null end))/24 '4-6 ESTper 5 min',
--(count(case when (datepart(hour, outc.datecompletedest) / 2) = 9 then 1 else null end))/24 '6-8 EST per 5 min',
--(count(case when (datepart(hour, outc.datecompletedest) / 2) = 10 then 1 else null end))/24 '8-10 EST per 5 min',
--(count(case when (datepart(hour, outc.datecompletedest) / 2) = 11 then 1 else null end))/24 '10-midnight EST per 5 min',
--(count(outc.datecompletedest))/288 'fullDay per 5 min'
FROM
    spa s WITH (nolock)
    LEFT JOIN brand b WITH (nolock)
    ON b.id = s.BrandID
    --left join [Order] o with (nolock)
    --    on o.SpaID = s.id
    LEFT JOIN (SELECT
        o.id
        ,o.spaid
        ,o.statusid
        ,o.IsDeleted
        ,o.SourceID
        ,dateadd(mi, datediff(mi, getutcdate(), getdate()), o.datecompletedutc) datecompletedEST
    FROM
        [order] o WITH (nolock)) oUTC
    ON outc.SpaID = s.id
WHERE
      s.StatusID = 4 /*Live Locations only*/
    AND s.IsTestAccount = 0 /*Exclude Internal Test Accounts*/
    AND s.IsDeleted = 0 /*Exclude Deleted Locations*/
    AND outc.StatusID = 2 /*Include only closed orders*/
    AND outc.IsDeleted = 0 /*Exclude deleted orders*/
    AND outc.datecompletedEST > '01-01-2023 00:00:00'
GROUP BY
     DATEPART(yyyy, outc.datecompletedest),
     DATEPART(mm, outc.datecompletedest),
datepart(dd,outc.datecompletedest),
	--(datepart(hour, o.DateCompleted) / 6),
DATENAME(dw, outc.datecompletedest)
ORDER BY
   year DESC,
   month DESC,
   day DESC
   --'totalordercount est'

**ePOS Usage Ratio**

In [ ]:
USE GolfNowOne
GO

SELECT
    (COUNT(CASE WHEN fs.EnableNewPOS = 1 THEN 1 END) / CAST(COUNT(*) AS decimal(10,2))) AS RatioOfEnableNewPOS
FROM [GolfNowOne].[dbo].[Spa] S WITH(NOLOCK)
INNER JOIN [GolfNowOne].[dbo].[SpaSettings] SS WITH(NOLOCK) ON SS.ID = S.SettingsID
INNER JOIN [GolfNowOne].[dbo].[FeatureSettings] FS WITH(NOLOCK) ON FS.ID = SS.FeatureSettingsID
WHERE s.[IsDeleted] = 0 AND s.BusinessTypeID = 45 AND s.StatusID = 4;


**Orders/Location**

In [1]:
USE GolfNowOne
GO

/* Total Sales and closed orders by 2 hour timeslot per day

CONVERTED TO EASTERN - Note that you'll have some orders that occurred late on west coast that roll into the next morning EST

Note that spas that are NOW terminated (or other status) but used to be live are not included. Need to figure out including legit ones but there is a lot of junk in there (in the terminated list, that is) and needs a cleanup before being included
*/
SELECT
    TOP 100
    s.AccountName
    ,s.Name
    ,count(1) as [Orders Completed between 10am-11am EST]

FROM
    spa s WITH (nolock)
    LEFT JOIN brand b WITH (nolock)
    ON b.id = s.BrandID
    --left join [Order] o with (nolock)
    --    on o.SpaID = s.id
    LEFT JOIN (SELECT
        o.id
        ,o.spaid
        ,o.statusid
        ,o.IsDeleted
        ,o.SourceID
        ,dateadd(mi, datediff(mi, getutcdate(), getdate()), o.datecompletedutc) datecompletedEST
    FROM
        [order] o WITH (nolock)) oUTC
    ON outc.SpaID = s.id
WHERE
      s.StatusID = 4 /*Live Locations only*/
    AND s.IsTestAccount = 0 /*Exclude Internal Test Accounts*/
    AND s.IsDeleted = 0 /*Exclude Deleted Locations*/
    AND outc.StatusID = 2 /*Include only closed orders*/
    AND outc.IsDeleted = 0 /*Exclude deleted orders*/
    AND outc.datecompletedEST > '12-01-2023 10:00:00'
    AND outc.datecompletedEST < '01-17-2024 11:00:00'
GROUP BY
     s.AccountName
     ,s.Name
ORDER BY
   count(1) DESC

**Reports By Location**

In [ ]:
USE GolfNowOne
GO

DECLARE @ReportID INT = 11524; --transaction rpt ID
DECLARE @ViewerTypeID INT = 2; --viewer type id for Spa/Location

SELECT
    S.[AccountName]
    ,R.[Name]
  FROM [GolfNowOne].[dbo].[Report] R WITH (NOLOCK)
  INNER JOIN [GolfNowOne].[dbo].[ReportViewerReport] RVR WITH (NOLOCK) ON RVR.ReportID = R.[ID]
  INNER JOIN [GolfNowOne].[dbo].[Spa] S WITH (NOLOCK) ON RVR.ViewerID = S.ID
  WHERE R.[ID] = @ReportID AND RVR.ViewerTypeID = @ViewerTypeID
  GROUP BY RVR.[ViewerID], R.[Name], S.AccountName


**Bluetooth Transaction Count per Location**

In [ ]:
USE GOLFNOWONE;

SELECT 
    S.[AccountName]
    ,COUNT(*) as 'Bluetooth Transactions'
FROM
    CreditCardTransaction CCT (NOLOCK)
    JOIN [Order] O (NOLOCK) ON CCT.OrderId = O.Id
    JOIN PaymentItem (NOLOCK) PIT ON PIT.ID = CCT.PaymentItemId
    JOIN dbo.[Spa] S (NOLOCK) ON S.ID = O.SpaId
    JOIN dbo.[CashRegister] t (NOLOCK) ON t.ID = O.CashRegisterId
    JOIN [dbo].[Areas] AS [a] (NOLOCK) ON [t].[AreaID] = [a].[ID]
    JOIN [dbo].[AreaSettings] AS [a0] (NOLOCK) ON [a].[SettingsID] = [a0].[ID]
    JOIN [dbo].[AreaPaymentSettings] AS [a1] (NOLOCK) ON [a0].[PaymentSettingsID] = [a1].[ID]
    LEFT JOIN CreditCardTransactionLog (NOLOCK) Lg ON Lg.CreditCardTransactionId = cct.ID
    LEFT JOIN [dbo].[GolfNowProcessorAccount] AS [g] (NOLOCK) ON [a1].[GNSwipeManualAccountId] = [g].[ID]
    LEFT JOIN [dbo].[GolfNowProcessorAccount] AS [g1] (NOLOCK) ON [a1].[GNPinPadAccountId] = [g].[ID]
    LEFT JOIN [dbo].[PinPadCache] AS [p] (NOLOCK) ON [t].[DefaultPinPadID] = [p].[ID]
    LEFT JOIN CreditCard (NOLOCK) CC ON CC.ID = PIT.CreditCardID
    LEFT JOIN CreditCardTransactionTypeLU (NOLOCK) TT ON CCT.TypeID = TT.ID
    LEFT JOIN EMVTransactionReceipt (NOLOCK) Recp ON Recp.ID = CCT.EMVTransactionReceiptID
    LEFT JOIN GolfNowProcessorTypeLU (NOLOCK) GNPT ON GNPT.ID = Recp.ProcessorType
WHERE 
     PIT.[IsBluetooth] = 1
GROUP BY S.[AccountName]


**Bluetooth Transactions for location**

In [ ]:
USE GOLFNOWONE;

SELECT
    TOP 50
    O.[ID]
    ,O.[DatePaidUTC]
    ,TT.[Name]
    ,GNPT.[Name] AS ProcessorType
    ,S.[accountname]
    ,a.[Name]
    ,t.[Name]
FROM
    CreditCardTransaction CCT (NOLOCK)
    JOIN [Order] O (NOLOCK) ON CCT.OrderId = O.Id
    JOIN PaymentItem (NOLOCK) PIT ON PIT.ID = CCT.PaymentItemId
    JOIN dbo.[Spa] S (NOLOCK) ON S.ID = O.SpaId
    JOIN dbo.[CashRegister] t (NOLOCK) ON t.ID = O.CashRegisterId
    JOIN [dbo].[Areas] AS [a] (NOLOCK) ON [t].[AreaID] = [a].[ID]
    JOIN [dbo].[AreaSettings] AS [a0] (NOLOCK) ON [a].[SettingsID] = [a0].[ID]
    JOIN [dbo].[AreaPaymentSettings] AS [a1] (NOLOCK) ON [a0].[PaymentSettingsID] = [a1].[ID]
    LEFT JOIN CreditCardTransactionLog (NOLOCK) Lg ON Lg.CreditCardTransactionId = cct.ID
    LEFT JOIN [dbo].[GolfNowProcessorAccount] AS [g] (NOLOCK) ON [a1].[GNSwipeManualAccountId] = [g].[ID]
    LEFT JOIN [dbo].[GolfNowProcessorAccount] AS [g1] (NOLOCK) ON [a1].[GNPinPadAccountId] = [g].[ID]
    LEFT JOIN [dbo].[PinPadCache] AS [p] (NOLOCK) ON [t].[DefaultPinPadID] = [p].[ID]
    LEFT JOIN CreditCard (NOLOCK) CC ON CC.ID = PIT.CreditCardID
    LEFT JOIN CreditCardTransactionTypeLU (NOLOCK) TT ON CCT.TypeID = TT.ID
    LEFT JOIN EMVTransactionReceipt (NOLOCK) Recp ON Recp.ID = CCT.EMVTransactionReceiptID
    LEFT JOIN GolfNowProcessorTypeLU (NOLOCK) GNPT ON GNPT.ID = Recp.ProcessorType
WHERE 
S.[ID] = 36960 AND PIT.[IsBluetooth] = 1
ORDER BY O.ID DESC, CCT.ID DESC